In [1]:
import math
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import model_from_json


In [2]:
DATASET_PATH = "../input/392-crypto-currency-pairs-at-minute-resolution/btcusd.csv"

In [3]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [4]:
df=pd.read_csv(DATASET_PATH)
data = df['close']

In [5]:
data = data.tolist()

In [6]:
len(data)

3229282

In [7]:
data = data[-1500000:]

In [8]:
len(data)

1500000

In [9]:
data = np.array(data)

In [10]:
print(data)

[ 7026.6         7033.9         7030.         ... 47113.
 47067.92745527 47069.        ]


In [11]:
print(np.min(data), np.max(data))

3215.3 58317.0


In [12]:
def make_data(data,train):
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    
    X_total = []
    y_total = []
    
    X_eql = []
    X_inc = []
    X_dec = []

    y_eql = []
    y_inc = []
    y_dec = []
    
    len_eql = 0
    len_inc = 0
    len_dec = 0
    
    for i in range(len(data)-401):
        
        X = data[i:i+400]
        mx = np.max(X)
        X = (X/mx)*0.9
        
        
        if data[i+400]>data[i+399]:
            y = [1,0,0]
            y_inc.append(y)
            len_inc+=1
            X_inc.append(X)
        if data[i+400]==data[i+399]:
            y = [0,1,0]
            y_eql.append(y)
            len_eql+=1
            X_eql.append(X)
        if data[i+400]<data[i+399]:
            y = [0,0,1]
            y_dec.append(y)
            len_dec+=1
            X_dec.append(X)
        
    length = min(len_inc,len_eql,len_dec)

    y_total = y_inc[:length] + y_eql[:length] + y_dec[:length]
    X_total = X_inc[:length] + X_eql[:length] + X_dec[:length]

    len_train = int(len(y_total)*train)
        
    X_train = X_total[:len_train]
    y_train = y_total[:len_train]
    
    X_val = X_total[-len_train:]
    y_val = y_total[-len_train:]
    
    X_train = np.array(X_train).reshape(-1,400,1)
    y_train = np.array(y_train).reshape(-1,3)
    X_val = np.array(X_val).reshape(-1,400,1)
    y_val = np.array(y_val).reshape(-1,3)


    
    return X_train, y_train, X_val, y_val


In [13]:
X_train, y_train, X_val, y_val = make_data(data,0.96)

In [14]:
y_train[0]

array([1, 0, 0])

In [15]:
X_train[0]

array([[0.85780431],
       [0.85732869],
       [0.85713357],
       [0.85701161],
       [0.85684088],
       [0.8580726 ],
       [0.85813358],
       [0.85732869],
       [0.85732869],
       [0.8583287 ],
       [0.85854822],
       [0.85884439],
       [0.85857431],
       [0.85867017],
       [0.86123118],
       [0.86062142],
       [0.8599019 ],
       [0.86010922],
       [0.85993848],
       [0.86013361],
       [0.86017019],
       [0.86215802],
       [0.86401171],
       [0.86315804],
       [0.86591417],
       [0.86541417],
       [0.86374341],
       [0.8630305 ],
       [0.86184095],
       [0.86285046],
       [0.86240193],
       [0.86184095],
       [0.86326779],
       [0.86307267],
       [0.86368243],
       [0.86236534],
       [0.86292633],
       [0.86265803],
       [0.86265803],
       [0.86265803],
       [0.8622312 ],
       [0.86142631],
       [0.86158485],
       [0.86132874],
       [0.86156045],
       [0.86259705],
       [0.86175516],
       [0.861

In [16]:
print(type(y_train[0]))

<class 'numpy.ndarray'>


In [17]:
X_train, y_train = shuffle(X_train, y_train)

In [18]:
y_train[10:20]

array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [19]:
model = Sequential()
model.add(LSTM(128,
          input_shape=(400, 1)))
model.add(Dense(100,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 79,763
Trainable params: 79,763
Non-trainable params: 0
_________________________________________________________________


In [21]:
# early_stop = EarlyStopping(monitor='val_loss', patience=20)

checkpoint = ModelCheckpoint("model_best.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')

In [22]:
history = model.fit(X_train, y_train, 
                    epochs=30, 
                    batch_size=50,
                    validation_data=(X_val,y_val),
                    callbacks=[checkpoint])

Epoch 1/30
12336/12338 [============================>.] - ETA: 0s - loss: 1.0970 - accuracy: 0.3472
Epoch 00001: val_accuracy improved from -inf to 0.34722, saving model to model_best.h5
12338/12338 [==============================] - 350s 28ms/step - loss: 1.0970 - accuracy: 0.3472 - val_loss: 1.1025 - val_accuracy: 0.3472
Epoch 2/30
12337/12338 [============================>.] - ETA: 0s - loss: 1.0968 - accuracy: 0.3497
Epoch 00002: val_accuracy did not improve from 0.34722
12338/12338 [==============================] - 353s 29ms/step - loss: 1.0968 - accuracy: 0.3497 - val_loss: 1.1543 - val_accuracy: 0.3056
Epoch 3/30
12337/12338 [============================>.] - ETA: 0s - loss: 1.0969 - accuracy: 0.3478
Epoch 00003: val_accuracy did not improve from 0.34722
12338/12338 [==============================] - 356s 29ms/step - loss: 1.0969 - accuracy: 0.3478 - val_loss: 1.1010 - val_accuracy: 0.3056
Epoch 4/30
12336/12338 [============================>.] - ETA: 0s - loss: 1.0969 - accura

In [23]:
model_json = model.to_json()
with open("model_final.json", "w") as json_file : 
    json_file.write(model_json)

In [24]:
model.save_weights("model_final.h5")

In [25]:
# with open("./model_final.json", "r") as fp:
#   model_test = model_from_json(fp.read())

# # Load weights to new model
# model_test.load_weights("./model_final.h5")

In [26]:
# df=pd.read_csv(DATASET_PATH)
# data_test = df['close']

In [27]:
# data_test = data_test.tolist()

In [28]:
# data_test = data_test[800000:1300000]

In [29]:
# data_test = np.array(data_test)

In [30]:
# plt.plot(range(len(data_test)),data_test)
# # plt.plot(range(len(train_data)),train_data)
# plt.show()

In [31]:
# X_train_test, y_train_test, X_val_test, y_val_test = make_data(data_test,0.95)

In [32]:
# all = 0
# counter = 0
# test = 0
# for i,y_guess in enumerate(model_test.predict(X_train_test)):

#     if y_train_test[i][0]>X_train_test[i][-1][0]:
#         all+=1
#         if y_guess[0]>X_train_test[i][-1][0]:
#             counter+=1
    
#     if y_train_test[i][0]==X_train_test[i][-1][0]:
#         test+=1

# print(counter/all)
# print(test/all)

In [33]:
# all = 0
# counter = 0
# for i,y_guess in enumerate(model_test.predict(X_train_test)):
    
#     if not y_train_test[i][0]==X_train_test[i][-1][0]:
#         all+=1

#     if y_train_test[i][0]<X_train_test[i][-1][0]:
#         if y_guess[0]<X_train_test[i][-1][0]:
#             counter+=1
#     if y_train_test[i][0]>X_train_test[i][-1][0]:
#         if y_guess[0]>X_train_test[i][-1][0]:
#             counter+=1

# print(counter/all)

In [34]:
# predict = model_test.predict(X_train_test)

In [35]:
# a=1
# for i,y_guess in enumerate(predict):
#     if y_guess[0]<X_train_test[i][-1][0]:
#         a *= (y_train_test[i][0]/X_train_test[i][-1][0])

In [36]:
# print(a)